# NGC Catalogのコンテナを実行する--TensorFlow-Docker

---

構築したOpenHPC環境でNGC Catalogの[TensorFlowコンテナ](https://ngc.nvidia.com/catalog/containers/nvidia:tensorflow)を実行します。

## 前提条件

このNotebookを実行するための前提条件を満たしていることを確認します。

以下のことを前提条件とします。
* 構築済のOpenHPC環境がある
* OpenHPC環境のマスターノードに対してSSHでログインできる
* 計算ノードでDockerが利用できること
* SlurmのGeneric Resource(GRES)としてGPUが登録されていること
* GPUがCUDA Compute Capability 5.2以上であること

> [AWS P2](https://aws.amazon.com/jp/ec2/instance-types/p2/)などのNVIDIA K80は[Compute Capability](https://developer.nvidia.com/cuda-gpus)が3.7なので、このNotebookで利用しているコンテナイメージを実行することができません。

マスターノードに対して SSH でログインできることを確認します。マスターノードのIPアドレスを指定してください。

In [ ]:
# (例)
# master_address = '172.30.XXX.xxx'

master_address = 

SSHでログインするユーザ名を指定してください。

In [ ]:
# (例)
# user = 'vcp'
# user = 'user00'

user = 

必要であればSSHの秘密鍵を指定してください。

In [ ]:
# (例)
# ssh_identity = '~/.ssh/id_rsa'

SSHでログインする際の引数を、変数に格納しておきます。

In [ ]:
# ユーザ名とホスト名
target = f'{user}@{master_address}'
print(target)

# SSHのコマンドライン引数
ssh_opts = f'-i {ssh_identity}' if 'ssh_identity' in vars() else ''
print(ssh_opts)

マスターノードに対してSSHでログインしてコマンドを実行してみます。

In [ ]:
!ssh {ssh_opts} {target} hostname

計算ノード数を指定してください。

In [ ]:
# (例)
# nodes = 1

nodes = 

Dockerを利用できることを確認します。

In [ ]:
!ssh {ssh_opts} {target} 'srun -N {nodes} docker info'

上のセルがエラーになった場合は Docker Engineがインストールされていないか docker を実行する権限が与えられていません。

以下のようなエラーが表示された場合は Docker Engine がインストールされていません。「071-DockerEngineのインストール.ipynb」を実行してDocker Engineを利用できるようにしてください。
```
slurmstepd: error: execve(): docker: No such file or directory
```

また、以下のようなエラーが表示された場合は docker を実行する権限が与えられていません。「051-ユーザの追加.ipynb」の「グループの追加」などを参考に`docker`グループへの追加を行ってください。
```
ERROR: Got permission denied while trying to connect to the Docker daemon socket at unix:///var/run/docker.sock: 
```

SlurmのGeneric Resource(GRES)としてGPUが登録されていることを確認します。

In [ ]:
!ssh {ssh_opts} {target} sinfo -N --Format=NodeHost,Gres | grep -w gpu

上のセルがエラーになった場合はSlurmにGPUがGRESとして登録されていません。「032-設定ファイルの編集-GRESの登録.ipynb」を実行してGRESの設定をしてください。

## TensorFlowコンテナの実行

TensorFlowコンテナでMNISTを実行してみます。

### コンテナイメージの取得

NGCカタログから[TensorFlow](https://ngc.nvidia.com/catalog/containers/nvidia:tensorflow)のコンテナイメージを取得します。

In [ ]:
ngc_version = '24.04'
docker_image = f'nvcr.io/nvidia/tensorflow:{ngc_version}-tf2-py3'

In [ ]:
!ssh {ssh_opts} {target} srun -N {nodes} docker pull {docker_image}

ユーザを追加したコンテナイメージを作成します。

In [ ]:
from pathlib import Path
from tempfile import TemporaryDirectory

!ssh {ssh_opts} {target} mkdir -p tensorflow-img
out = !ssh {ssh_opts} {target} id -u
uid = out[0]
with TemporaryDirectory() as workdir:
    dockerfile = Path(workdir) / 'Dockerfile'
    with dockerfile.open(mode='w') as f:
        f.write(f'''
FROM {docker_image}

ARG USER={user}
RUN useradd -m ${{USER}} -u {uid}
WORKDIR /home/${{USER}}
USER {user}
''')
    !cat {dockerfile}
    !scp {ssh_opts} {dockerfile} {target}:tensorflow-img

!ssh {ssh_opts} {target} srun -N {nodes} docker build -t tensorflow-{user} tensorflow-img

### 準備

MNISTのデータやスクリプトを準備します。

まず、データやスクリプトを配置するディレクトリを作成します。

In [ ]:
work_dir = 'tensorflow'
!ssh {ssh_opts} {target} mkdir -p {work_dir}

データをダウンロードするためのスクリプトを配置します。

In [ ]:
!scp {ssh_opts} template/tensorflow/download_mnist.py {target}:{work_dir}

TensorFlowの[Quickstart for beginners](https://www.tensorflow.org/tutorials/quickstart/beginner)で実行しているのと同等のスクリプトをOpenHPC環境に配置します。

In [ ]:
!scp {ssh_opts} template/tensorflow/mnist_classify.py {target}:{work_dir}

取得したスクリプトの内容を先頭部分を表示してみます。

In [ ]:
!ssh {ssh_opts} {target} head {work_dir}/mnist_classify.py

### コンテナからGPUが利用できることを確認する

Singularityで実行したコンテナ環境からGPUを利用できることを確認します。

GPUを利用できるかをチェックするスクリプトを配置します。

In [ ]:
!scp {ssh_opts} template/tensorflow/check_gpu.py {target}:{work_dir}

コンテナを実行できるようにするためにホームディレクトリのパーミッションを変更します。

In [ ]:
!ssh {ssh_opts} {target} chmod 755 .

GPU利用の可否をチェックするスクリプトを実行します。次のセルを実行してエラーとならないことを確認してください。

In [ ]:
!ssh {ssh_opts} {target} srun -N {nodes} --gpus-per-node=1 \
    docker run --gpus all --ipc=host -v /home/{user}:/home/{user} --rm tensorflow-{user} \
    python {work_dir}/check_gpu.py

### ジョブの実行

MNISTのスクリプトをSlurmのジョブとして実行します。

ジョブの実行スクリプトを作成します。

In [ ]:
from tempfile import TemporaryDirectory
from pathlib import Path

with TemporaryDirectory() as workdir:
    batch_file = Path(workdir) / 'tensorflow_mnist_docker.job'
    with batch_file.open(mode='w') as f:
        f.write(f'''#!/bin/bash
        
#SBATCH -J tensorflow-mnist-docker         # create a short name for your job
#SBATCH -o tensorflow-mnist-docker.%j.out  # Name of stdout output file (%j expands to jobId)
#SBATCH -N 1                               # Total number of nodes requested
#SBATCH -n 1                               # Total number of across all nodes
#SBATCH --gres=gpu:1                       # number of gpus per node
#SBATCH -t 00:10:00                        # Run time (hh:mm:ss)

docker run -td --gpus all -v /home/{user}:/home/{user} --rm --ipc=host --net=host --name tensorflow-{user} tensorflow-{user}
docker exec -t -u {user} -w $HOME/{work_dir} tensorflow-{user} python3 download_mnist.py
docker exec -t -u {user} -w $HOME/{work_dir} tensorflow-{user} python3 mnist_classify.py
docker stop tensorflow-{user}
''')
    !cat {batch_file}
    !scp {ssh_opts} {str(batch_file)} {target}:{work_dir}

ジョブを実行する前のキューの状態を確認します。

In [ ]:
!ssh {ssh_opts} {target} squeue

ノードのGPU利用状況を確認します。

In [ ]:
!ssh {ssh_opts} {target} sinfo --Node --Format=NodeHost,Gres,GresUsed

ジョブを実行します。

In [ ]:
!ssh {ssh_opts} {target} bash -l -c \
    "'cd {work_dir} && sbatch tensorflow_mnist_docker.job'"

ジョブの実行状況を確認します。

In [ ]:
!ssh {ssh_opts} {target} squeue

ノードのGPU利用状況を確認します。`GRES_USED` の欄でノードのGPU利用状況を確認してください。

In [ ]:
!ssh {ssh_opts} {target} sinfo --Node --Format=NodeHost,Gres,GresUsed

ジョブが完了するまで数分かかります。ジョブ実行中のCPU, メモリ, GPUなどの利用状況は VCC の Grafana で確認することができます。

ジョブの完了後に次のセルを実行してください。ジョブの出力結果が確認できます。

In [ ]:
!ssh {ssh_opts} {target} bash -l -c \
    "'cd {work_dir} && tail tensorflow-mnist*.out'"

ジョブの実行が完了していれば出力結果の末尾に以下のような内容が表示されます。

```
1875/1875 [==============================] - 2s 1ms/step - loss: 0.1098 - accuracy: 0.9664
Epoch 4/5
1875/1875 [==============================] - 2s 1ms/step - loss: 0.0891 - accuracy: 0.9724
Epoch 5/5
1875/1875 [==============================] - 2s 1ms/step - loss: 0.0748 - accuracy: 0.9755
313/313 - 0s - loss: 0.0839 - accuracy: 0.9748

Test accuracy: 0.9747999906539917
```